# Excited delirium analysis

Looking at case involving the offical cause of death called Excited delirium. This relates to the story [In fatal struggles with police, a controversial killer is often blamed](http://www.mystatesman.com/news/state--regional/fatal-struggles-with-police-controversial-killer-often-blamed/5NAcIthmQubqObTSDnVdKI/)

The field to look at is **`cause_of_death`**, where the ID for [Excited delirium is 1](http://data.statesman.com/admin/custody_deaths/cause/1/change/).

## Findings

- [Excited delirium as cause of death](#Excited-delirium-as-cause-of-death)
- [Excited delirium cases by year](#Excited-delirium-cases-by-year)
- [Excited delirium cases by race](#Excited-delirium-cases-by-race)
- [Other 'delirium' cases](#Other-delirium-causes)

  - compare to overall (available in [Overall data stats](https://github.com/statesman/custody-deaths-analysis/blob/master/notebooks/Overall%20data%20stats.ipynb))




In [1]:
# imports
import agate
import re
import warnings
warnings.filterwarnings('ignore')

## Get and set up the data

Browse data sets available then import deaths.

### Import main table

In [2]:
# sets data types on fields agate got wrong
specified_data_types = {
    'tracked_cause': agate.Text(),
    'offense': agate.Text(),
    'case_study': agate.Text(),
    'official_discipline': agate.Text()
}

# this pulls the deaths file that is exported in the File processing notebook
deaths = agate.Table.from_csv('../exports/deaths_latest.csv', column_types=specified_data_types)

print(deaths)

| column               | data_type |
| -------------------- | --------- |
| id                   | Number    |
| ag_report_url        | Text      |
| first_name           | Text      |
| middle_name          | Text      |
| last_name            | Text      |
| suffix               | Text      |
| slug                 | Text      |
| race                 | Text      |
| gender               | Text      |
| date_of_birth        | Date      |
| date_of_death        | Date      |
| age                  | Number    |
| agency               | Number    |
| restrained           | Boolean   |
| tazed                | Boolean   |
| times_tazed          | Number    |
| pepper_sprayed       | Boolean   |
| official_discipline  | Text      |
| grand_jury_result    | Text      |
| mental_health_issues | Boolean   |
| manner_of_death      | Text      |
| drug_intoxication    | Boolean   |
| cause_of_death       | Text      |
| tracked_cause        | Text      |
| offense              | Text      |
|

### Function to parse tracked_cause

In [3]:
# functon to check if tracked_cause includes 1 - Excited delirium
ed = ['1']

# This is the evaluation function, used to compare offenses in a row
# to the list those that include excited delirium.
def parse_tracked_cause(row):
    if row is None:
        return False
    myList = row.split(',')
    for item in myList:
        if item in ed:
            return True



## Excited delirium as cause of death

In [4]:
# this filters our death_offenses table based on the 
# value we are looking for in the column we are looking
deaths_ed = deaths.where(lambda row: parse_tracked_cause(row['tracked_cause']))

# Number of deaths that are Excited delirium
print('Number of deaths that are Excited delirium: {}\n'.format(
        len(deaths_ed))
      )

# prints the deaths with just minor offenses
deaths_ed.select(['first_name',
                  'middle_name',
                  'last_name',
                  'date_of_death',
                  'tracked_cause']).order_by('date_of_death').print_table(max_rows=None)


Number of deaths that are Excited delirium: 50

| first_name  | middle_name | last_name | date_of_death | tracked_cause |
| ----------- | ----------- | --------- | ------------- | ------------- |
| Curtis      | Lee         | Lewis     |    2005-01-10 | 1,2           |
| Hector      |             | Rodriguez |    2005-05-07 | 1             |
| Joseph      |             | Cain      |    2005-05-17 | 1             |
| Al          | Dwaun       | Jackson   |    2005-06-06 | 1             |
| Robert      | Earl        | Williams  |    2005-06-14 | 1,36          |
| Gilberto    |             | Limon     |    2005-09-19 | 1             |
| Carl        | James       | O'Neal    |    2005-11-10 | 1             |
| Charles     | Edward      | Boll      |    2005-11-14 | 1             |
| Joe         | Ernest      | Ramos     |    2006-01-27 | 1             |
| Jose        |             | Romero    |    2006-04-24 | 1,36          |
| Ben         | Tarell      | Miller    |    2006-05-19 | 1     

## Excited delirium cases by year

In [5]:
# add a year column based on year of date_of_death
deaths_ed_year = deaths_ed.compute([
    ('Year', agate.Formula(agate.Text(), lambda r: r['date_of_death'].year))
])

# pivot based on year
deaths_ed_year_pivot = deaths_ed_year.pivot('Year').order_by('Year')

# print the pivot as bars
deaths_ed_year_pivot.print_bars('Year', 'Count', width=60)

Year Count
2005     8 ▓░░░░░░░░░░░░░░░░░░░                             
2006    12 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                   
2007     9 ▓░░░░░░░░░░░░░░░░░░░░░░                          
2008     3 ▓░░░░░░░                                         
2009     2 ▓░░░░░                                           
2011     6 ▓░░░░░░░░░░░░░░                                  
2012     2 ▓░░░░░                                           
2013     4 ▓░░░░░░░░░░                                      
2014     2 ▓░░░░░                                           
2015     2 ▓░░░░░                                           
           +-----------+-----------+-----------+-----------+
           0           5          10          15          20


## Excited delirium cases by race

In [6]:
# pivot deaths_ed by race_name
deaths_ed_pivot = deaths_ed.pivot('race_name').order_by('race_name')

# print table of pivot
print('Excited delirium race breakdown\n')
deaths_ed_pivot.print_table()

# print chart based on excited delirium by race
print('\nExcited delirium race breakdown: Bars\n')
deaths_ed_pivot.print_bars('race_name',
                           'Count',
                           width=60,
                           domain=['0', '200'])

print('\nCompare to overall race breakdown\n')
# pivot deaths by race
race_pivot = deaths.pivot('race_name')

# print bar chart of pivot
race_pivot.print_bars('race_name',
                      'Count',
                      width=60,
                      domain=['0', '200'])


print('\nCompare percentage of blacks in ED cohort vs overall, based on total {} cases.\n'.format(
        len(deaths)
    ))

# percent of ed cohort that is black
black_ed_count = deaths_ed.aggregate(agate.Count('race', 'b'))
black_count = deaths.aggregate(agate.Count('race', 'b'))

print('Percent of blacks:\n Excited delirium cohort: {:.1%}\n Overall: {:.1%}\n'.format(
        black_ed_count / len(deaths_ed),
        black_count / len(deaths)
    ))

# percent of ed cohort that is latino
latino_ed_count = deaths_ed.aggregate(agate.Count('race', 'h'))
latino_count = deaths.aggregate(agate.Count('race', 'h'))

print('Percent of Latinos:\n Excited delirium cohort: {:.1%}\n Overall: {:.1%}\n'.format(
        latino_ed_count / len(deaths_ed),
        latino_count / len(deaths)
    ))

# percent of ed cohort that is white
white_ed_count = deaths_ed.aggregate(agate.Count('race', 'w'))
white_count = deaths.aggregate(agate.Count('race', 'w'))

print('Percent of whites:\n Excited delirium cohort: {:.1%}\n Overall: {:.1%}\n'.format(
        white_ed_count / len(deaths_ed),
        white_count / len(deaths)
    ))

print('Percent minority:\n Excited delirium cohort: {:.1%}\n Overall: {:.1%}\n'.format(
        (black_ed_count + latino_ed_count) / len(deaths_ed),
        (black_count + latino_count) / len(deaths)
    ))


Excited delirium race breakdown

| race_name       | Count |
| --------------- | ----- |
| Black           |    19 |
| Hispanic/Latino |    23 |
| White           |     8 |

Excited delirium race breakdown: Bars

race_name       Count
Black              19 ▓░░░░                                 
Hispanic/Latino    23 ▓░░░░                                 
White               8 ▓░                                    
                      +--------+--------+---------+--------+
                      0       50       100       150     200

Compare to overall race breakdown

race_name       Count
White              93 ▓░░░░░░░░░░░░░░░░░                    
Black             101 ▓░░░░░░░░░░░░░░░░░░░                  
Hispanic/Latino    91 ▓░░░░░░░░░░░░░░░░░                    
Asian               4 ▓░                                    
                      +--------+--------+---------+--------+
                      0       50       100       150     200

Compare percentage of blacks in ED 

## By Agency

In [7]:
# deaths_ed.pivot('agency').print_table()
agencies = agate.Table.from_csv('../data_raw/agency_table.csv')
deaths_ed_agency = deaths_ed.join(agencies, 'agency', 'id').rename(column_names = {'name': 'agency_name'})
deaths_ed_agency.pivot('agency_name').order_by('Count', reverse=True).print_table(max_column_width=None)

| agency_name                       | Count |
| --------------------------------- | ----- |
| San Antonio PD                    |    12 |
| Dallas PD                         |     9 |
| Corpus Christi PD                 |     4 |
| Texas Department of Public Safety |     2 |
| Waco PD                           |     2 |
| Hidalgo County Sheriffs Dept.     |     1 |
| Van Zandt Sheriff's Dept.         |     1 |
| Medina County SO                  |     1 |
| Gregg County SO                   |     1 |
| Dimmit County SO                  |     1 |
| Dallas County SO                  |     1 |
| Cameron County SO                 |     1 |
| Aransas County SO                 |     1 |
| Weatherford PD                    |     1 |
| Victoria PD                       |     1 |
| Temple PD                         |     1 |
| Rockport PD                       |     1 |
| Rio Grande City PD                |     1 |
| Lancaster PD                      |     1 |
| La Marque PD                    

## Excited delirium and tasers

In [8]:
# of the ed cases which have tazed = yes
deaths_ed.pivot('tazed').print_table()

| tazed | Count |
| ----- | ----- |
| False |    36 |
|  True |    14 |


## Other delirium causes

These cases have the term "psychotic delirium" in their cause of dath.

In [9]:
#.select(['first_name','last_name'])

causes_list_delirium = deaths.where(lambda row: re.search(
        '(?i)psychotic delirium',
        str(row['cause_of_death'])
    ))

print('Number of cases: {}\n'.format(
    len(causes_list_delirium)))

causes_list_delirium.select(['first_name','last_name']).print_table(max_column_width=None, max_rows=None)

Number of cases: 2

| first_name | last_name |
| ---------- | --------- |
| Joel       | Casey     |
| Daryl      | Kelley    |
